In [79]:
ROBBER = "R"
POLICE = "P"
BANK = "B"
EMPTY = "*"

grid = [
    [ROBBER,EMPTY,EMPTY,EMPTY],
    [EMPTY,BANK,EMPTY,EMPTY],
    [EMPTY,EMPTY,EMPTY,EMPTY],
    [EMPTY,EMPTY,EMPTY, POLICE]
]

for row in grid:
    print(' '.join(row))


R * * *
* B * *
* * * *
* * * P


In [80]:
class State:
    
    def __init__(self, grid, robber_pos, police_pos):
        self.grid = grid
        self.robber_pos = robber_pos
        self.police_pos = police_pos
    """    
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.car_pos == other.car_pos and self.police_pos == other.police_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.car_pos) + str(self.police_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, car_pos={self.car_pos}, police_pos={self.police_pos})"
    """

In [81]:
STAY = 0
UP = 1
DOWN = 2 
LEFT = 3
RIGHT = 4

ACTIONS = [STAY, UP, DOWN, LEFT, RIGHT]

REWARD_BANK = 1
REWARD_EMPTY = -1
REWARD_CAUGHT = -10

START_ROBBER = [0,0]
START_POLICE = [3,3]

In [82]:
start_state = State(grid=grid, robber_pos=START_ROBBER, police_pos=START_POLICE)
print(start_state.grid[0][0])

R


In [118]:
from copy import deepcopy

def act(state,action_robber,action_police):
    # state_actor_pos = state.robber_pos for robber or state.police_pos
    def new_actor_pos(state,state_actor_pos,action):
        p = deepcopy(state_actor_pos)
        if action == UP:
            print('UP')
            p[0] = max(0,p[0]-1)
        elif action == DOWN:
            print('down')
            p[0] = min(len(state.grid)-1, p[0]+1)
        elif action == LEFT:
            print('LEFT')
            p[1] = max(0, p[1]-1)
        elif action == RIGHT:
            print('RIGHT')
            p[1] = min(len(state.grid[0])-1, p[1]+1)
        elif state_actor_pos == deepcopy(state.robber_pos):
            if action == STAY:
                print('STAY')
                p = p
            else:
                print('error robber')
                raise ValueError(f"Unknown action {action}")
        else:
            print('error police')
            raise ValueError(f"Unknown action {action}")
        return p
    
    pos_robber = new_actor_pos(state,state.robber_pos,action_robber)
    pos_police = new_actor_pos(state,state.police_pos,action_police)
    
    new_grid = deepcopy(state.grid)
    
    old_pos_robber = state.robber_pos
    old_pos_police = state.police_pos
    
    grid_item_robber = state.grid[pos_robber[0]][pos_robber[1]]
    grid_item_police = state.grid[pos_police[0]][pos_police[1]]
    old_grid_item_robber = state.grid[old_pos_robber[0]][old_pos_robber[1]]
    old_grid_item_police = state.grid[old_pos_police[0]][old_pos_police[1]]
    
    if pos_robber == pos_police:
        print('1')
        print('OVER')
        reward = REWARD_CAUGHT
        is_over = True
        if not old_pos_robber == pos_robber:
            if old_grid_item_robber == BANK+ROBBER:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
                new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            elif grid_item_robber == BANK:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
                new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER 
            else:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
                new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        if old_grid_item_police == BANK+POLICE:
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] += POLICE
        else:
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] += POLICE
    
    elif pos_robber == old_pos_police and pos_police == old_pos_robber:
        print('2')
        if grid_item_robber == BANK+POLICE:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK+ROBBER
        elif grid_item_police == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK+POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = ROBBER
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = ROBBER
    
    elif pos_police == old_pos_robber and pos_robber != old_pos_police:
        print('3')
        if old_grid_item_police == BANK+POLICE:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        elif old_grid_item_robber == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        elif grid_item_robber == BANK:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
    
    elif pos_robber == old_pos_police and pos_police != old_pos_robber:
        print('4')
        if old_grid_item_robber == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        elif old_grid_item_police == BANK+POLICE:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        elif grid_item_police == BANK:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        
    else:
        print('5')
        if grid_item_police == BANK:
            print("Polis bank")
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
        elif grid_item_robber == BANK:
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif grid_item_robber == BANK+ROBBER:
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif grid_item_robber == ROBBER:
            print('HÄÄR')
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif old_grid_item_robber == BANK+ROBBER:
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif old_grid_item_police == BANK+POLICE:
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
        else:
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
    
  
    #return grid_item_robber, old_grid_item_robber, grid_item_police, old_grid_item_police #new_grig#State(grid=new_grid, robber_pos=p, police_pos=[3,3])    
    return State(grid=new_grid,robber_pos=pos_robber, police_pos=pos_police)

for row in grid:
    print(' '.join(row))
    
a = act(start_state, DOWN, UP)
#print(a)
for row in a.grid:
    print(' '.join(row))

b = act(a, RIGHT, LEFT)
for row in b.grid:
    print(' '.join(row))
    
c = act(b, STAY, LEFT)
for row in c.grid:
    print(' '.join(row))

d = act(c, STAY, UP)
for row in d.grid:
    print(' '.join(row))

e = act(d, RIGHT, UP)
for row in e.grid:
    print(' '.join(row))
    
f = act(e, STAY, UP)
for row in f.grid:
    print(' '.join(row))
    
g = act(f, RIGHT, LEFT)
for row in g.grid:
    print(' '.join(row))




R * * *
* B * *
* * * *
* * * P
down
UP
5
* * * *
R B * *
* * * P
* * * *
RIGHT
LEFT
5
* * * *
* BR * *
* * P *
* * * *
STAY
LEFT
5
* * * *
* BR * *
* P * *
* * * *
STAY
UP
1
OVER
* * * *
* BRP * *
* * * *
* * * *
RIGHT
UP
5
* P * *
* * R *
* * * *
* * * *
STAY
UP
5
HÄÄR
* * * *
* * R *
* * * *
* * * *
RIGHT
LEFT
5
P * * *
* * * R
* * * *
* * * *
